# Day 09 - claude

In [2]:
def parse_disk_map(disk_map):
    """Parse the disk map string into a list of integers."""
    return [int(char) for char in disk_map.strip()]

def create_blocks(lengths):
    """
    Create initial blocks array where each element is either None (free space) 
    or a file ID number. Memory efficient as we only create one array.
    """
    blocks = []
    file_id = 0
    for i, length in enumerate(lengths):
        if i % 2 == 0:  # File
            blocks.extend([file_id] * length)
            file_id += 1
        else:  # Free space
            blocks.extend([None] * length)
    return blocks

def find_moves(blocks):
    """
    Generator that yields (from_idx, to_idx) for each move that needs to be made.
    This avoids storing all intermediate states.
    """
    while True:
        # Find leftmost free space
        to_idx = None
        for i, block in enumerate(blocks):
            if block is None:
                to_idx = i
                break
        if to_idx is None:
            break

        # Find rightmost file block after the free space
        from_idx = None
        for i in range(len(blocks) - 1, to_idx - 1, -1):
            if blocks[i] is not None:
                from_idx = i
                break
        
        if from_idx is None or from_idx < to_idx:
            break

        yield from_idx, to_idx

def calculate_checksum(blocks):
    """Calculate the checksum based on position * file ID."""
    return sum(pos * file_id 
              for pos, file_id in enumerate(blocks) 
              if file_id is not None)

def solve_disk_compaction(input_file):
    """
    Main function to solve the disk compaction problem.
    Memory efficient implementation that only stores current state.
    """
    # Read and parse input
    with open(input_file, 'r') as f:
        disk_map = f.read().strip()
    
    # Create initial blocks
    lengths = parse_disk_map(disk_map)
    blocks = create_blocks(lengths)
    
    # Perform moves
    for from_idx, to_idx in find_moves(blocks):
        # Move one block
        blocks[to_idx] = blocks[from_idx]
        blocks[from_idx] = None
    
    # Calculate and return checksum
    return calculate_checksum(blocks)

if __name__ == "__main__":
    checksum = solve_disk_compaction('input.txt')
    print(f"Final checksum: {checksum}")

Final checksum: 6430446922192


## Part 2

In [3]:
def parse_disk_map(disk_map):
    """Parse the disk map string into a list of integers."""
    return [int(char) for char in disk_map.strip()]

def create_blocks(lengths):
    """
    Create initial blocks array where each element is either None (free space) 
    or a file ID number.
    """
    blocks = []
    file_id = 0
    for i, length in enumerate(lengths):
        if i % 2 == 0:  # File
            blocks.extend([file_id] * length)
            file_id += 1
        else:  # Free space
            blocks.extend([None] * length)
    return blocks

def get_file_info(blocks):
    """
    Get information about each file's position and length.
    Returns dict mapping file_id to (start_pos, length).
    """
    file_info = {}
    current_file = None
    start_pos = None
    
    for pos, block in enumerate(blocks):
        if block != current_file:
            # If we were tracking a file, record its info
            if current_file is not None and current_file not in file_info:
                file_info[current_file] = (start_pos, pos - start_pos)
            # Start tracking new file
            current_file = block
            start_pos = pos
    
    # Handle the last file
    if current_file is not None and current_file not in file_info:
        file_info[current_file] = (start_pos, len(blocks) - start_pos)
    
    return file_info

def find_leftmost_space(blocks, start_idx, required_length):
    """Find the leftmost contiguous free space that can fit the required length."""
    current_length = 0
    start_pos = None
    
    for i in range(len(blocks)):
        if blocks[i] is None:
            if start_pos is None:
                start_pos = i
            current_length += 1
            if current_length >= required_length:
                return start_pos
        else:
            start_pos = None
            current_length = 0
    
    return None

def move_file(blocks, file_id, from_pos, length, to_pos):
    """Move a whole file from one position to another."""
    # Clear original position
    for i in range(from_pos, from_pos + length):
        blocks[i] = None
    # Place file in new position
    for i in range(to_pos, to_pos + length):
        blocks[i] = file_id

def calculate_checksum(blocks):
    """Calculate the checksum based on position * file ID."""
    return sum(pos * file_id 
              for pos, file_id in enumerate(blocks) 
              if file_id is not None)

def solve_disk_compaction_part2(input_file):
    """
    Solve part 2 of the disk compaction problem.
    Move whole files in order of decreasing file ID.
    """
    # Read and parse input
    with open(input_file, 'r') as f:
        disk_map = f.read().strip()
    
    # Create initial blocks
    lengths = parse_disk_map(disk_map)
    blocks = create_blocks(lengths)
    
    # Get information about all files
    file_info = get_file_info(blocks)
    
    # Process files in order of decreasing file ID
    for file_id in sorted(file_info.keys(), reverse=True):
        start_pos, length = file_info[file_id]
        
        # Find leftmost suitable free space
        new_pos = find_leftmost_space(blocks, 0, length)
        
        # If we found a suitable position and it's to the left of the current position
        if new_pos is not None and new_pos < start_pos:
            move_file(blocks, file_id, start_pos, length, new_pos)
    
    # Calculate and return checksum
    return calculate_checksum(blocks)

if __name__ == "__main__":
    checksum = solve_disk_compaction_part2('input.txt')
    print(f"Final checksum: {checksum}")

Final checksum: 6460170593016
